### Create Antecedents and Consequent
- Import membership functions from `fuzzy_memberships.py` (universes and sets).
- Define 5 antecedents: BMI, Age, HighBP, Smoker, PhysActivity.
- Define 1 consequent: Risk (use `defuzzify_method='centroid'`).
- Attach membership functions to each variable:
  - BMI: under, healthy, over, obese
  - Age: young, middle, old
  - HighBP: normal, high
  - Smoker: no, yes
  - PhysActivity: inactive, active
  - Risk: vlow, low, med, high, vhigh
- Sanity check: each variable has all labels and names match the imports.

In [1]:
import pandas as pd
import numpy as np
import skfuzzy.control as ctrl
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix


# --- Import all universe variables and MFs from the external file ---
try:
    from fuzzy_memberships import (
        bmi_universe, age_universe, binary_universe, risk_universe,
        bmi_under, bmi_healthy, bmi_over, bmi_obese,
        age_young, age_middle, age_old,
        bp_normal, bp_high,
        smoke_no, smoke_yes,
        act_inactive, act_active,
        risk_vlow, risk_low, risk_med, risk_high, risk_vhigh
    )
    print("Fuzzy membership functions imported successfully from fuzzy_memberships.py.")
except ImportError:
    print("Error: Could not import from fuzzy_memberships.py. Ensure the file exists.")
    exit()

# --- Load Cleaned Data ---
try:
    df = pd.read_csv("../data/diabetes_clean.csv")
    print("Cleaned data loaded successfully.")
except FileNotFoundError:
    print("Error: 'diabetes_clean.csv' not found. Ensure the file path is correct.")
    exit()

Fuzzy membership functions imported successfully from fuzzy_memberships.py.
Cleaned data loaded successfully.


In [2]:
# D3 — Create Antecedents and Consequent
bmi = ctrl.Antecedent(bmi_universe, 'BMI')
age = ctrl.Antecedent(age_universe, 'Age')
highbp = ctrl.Antecedent(binary_universe, 'HighBP')
smoker = ctrl.Antecedent(binary_universe, 'Smoker')
physact = ctrl.Antecedent(binary_universe, 'PhysActivity')

risk = ctrl.Consequent(risk_universe, 'Risk', defuzzify_method='centroid')

bmi['under'] = bmi_under
bmi['healthy'] = bmi_healthy
bmi['over'] = bmi_over
bmi['obese'] = bmi_obese

age['young'] = age_young
age['middle'] = age_middle
age['old'] = age_old

highbp['normal'] = bp_normal
highbp['high'] = bp_high

smoker['no'] = smoke_no
smoker['yes'] = smoke_yes

physact['inactive'] = act_inactive
physact['active'] = act_active

risk['vlow'] = risk_vlow
risk['low'] = risk_low
risk['med'] = risk_med
risk['high'] = risk_high
risk['vhigh'] = risk_vhigh

In [3]:
# D4 — Define Rule Base (20 rules) and Build System
all_rules = []

# VLOW/LOW
all_rules.append(ctrl.Rule(bmi['healthy'] & age['young'] & highbp['normal'] & smoker['no'] & physact['active'], risk['vlow']))
all_rules.append(ctrl.Rule(bmi['under'] & highbp['normal'] & physact['active'] & smoker['no'], risk['vlow']))
all_rules.append(ctrl.Rule(bmi['healthy'] & age['middle'] & highbp['normal'] & physact['active'], risk['low']))
all_rules.append(ctrl.Rule(bmi['healthy'] & age['young'] & highbp['normal'] & smoker['yes'], risk['low']))
all_rules.append(ctrl.Rule(bmi['over'] & age['young'] & highbp['normal'] & physact['active'], risk['low']))

# MEDIUM
all_rules.append(ctrl.Rule(bmi['over'] & physact['inactive'] & highbp['normal'], risk['med']))
all_rules.append(ctrl.Rule(bmi['obese'] & age['young'] & physact['active'], risk['med']))
all_rules.append(ctrl.Rule(highbp['high'] & age['young'] & smoker['no'] & physact['active'], risk['med']))
all_rules.append(ctrl.Rule(bmi['healthy'] & age['old'] & physact['inactive'], risk['med']))
all_rules.append(ctrl.Rule(smoker['yes'] & physact['inactive'] & highbp['normal'], risk['med']))
all_rules.append(ctrl.Rule(bmi['over'] & age['middle'] & highbp['normal'] & smoker['no'], risk['med']))
all_rules.append(ctrl.Rule(highbp['high'] & age['middle'] & bmi['healthy'], risk['med']))

# HIGH
all_rules.append(ctrl.Rule(bmi['obese'] & age['middle'] & highbp['normal'], risk['high']))
all_rules.append(ctrl.Rule(highbp['high'] & smoker['yes'] & age['middle'], risk['high']))
all_rules.append(ctrl.Rule(bmi['over'] & age['old'] & physact['inactive'], risk['high']))
all_rules.append(ctrl.Rule(bmi['obese'] & physact['inactive'], risk['high']))
all_rules.append(ctrl.Rule(highbp['high'] & age['old'] & smoker['no'], risk['high']))

# VHIGH
all_rules.append(ctrl.Rule(bmi['obese'] & highbp['high'] & age['old'], risk['vhigh']))
all_rules.append(ctrl.Rule(bmi['obese'] & highbp['high'] & smoker['yes'], risk['vhigh']))
all_rules.append(ctrl.Rule(age['old'] & highbp['high'] & physact['inactive'], risk['vhigh']))

print(f"Total number of rules created: {len(all_rules)}")

risk_ctrl = ctrl.ControlSystem(all_rules)
risk_sim = ctrl.ControlSystemSimulation(risk_ctrl)
print("Fuzzy Control System and Simulation object created.")

print("\n--- Testing Samples ---")
risk_sim.input['BMI'] = 0.35
risk_sim.input['Age'] = 0.1
risk_sim.input['HighBP'] = 0.0
risk_sim.input['Smoker'] = 0.0
risk_sim.input['PhysActivity'] = 1.0
risk_sim.compute()
print(f"Case 1 (Optimal Health - VLOW Expected) Output: {risk_sim.output['Risk']:.4f}")

risk_sim.input['BMI'] = 0.5
risk_sim.input['Age'] = 0.5
risk_sim.input['HighBP'] = 0.0
risk_sim.input['Smoker'] = 1.0
risk_sim.input['PhysActivity'] = 1.0
risk_sim.compute()
print(f"Case 2 (Overweight Smoker - LOW Expected) Output: {risk_sim.output['Risk']:.4f}")

risk_sim.input['BMI'] = 0.8
risk_sim.input['Age'] = 0.2
risk_sim.input['HighBP'] = 0.0
risk_sim.input['Smoker'] = 0.0
risk_sim.input['PhysActivity'] = 1.0
risk_sim.compute()
print(f"Case 3 (Obese, Young/Active - MED Expected) Output: {risk_sim.output['Risk']:.4f}")

risk_sim.input['BMI'] = 0.35
risk_sim.input['Age'] = 0.8
risk_sim.input['HighBP'] = 1.0
risk_sim.input['Smoker'] = 0.0
risk_sim.input['PhysActivity'] = 0.0
risk_sim.compute()
print(f"Case 4 (High BP, Old, Inactive - HIGH Expected) Output: {risk_sim.output['Risk']:.4f}")

risk_sim.input['BMI'] = 0.95
risk_sim.input['Age'] = 0.95
risk_sim.input['HighBP'] = 1.0
risk_sim.input['Smoker'] = 1.0
risk_sim.input['PhysActivity'] = 0.0
risk_sim.compute()
print(f"Case 5 (Triple Threat - VHIGH Expected) Output: {risk_sim.output['Risk']:.4f}")

Total number of rules created: 20
Fuzzy Control System and Simulation object created.

--- Testing Samples ---
Case 1 (Optimal Health - VLOW Expected) Output: 0.0700
Case 2 (Overweight Smoker - LOW Expected) Output: 0.2500
Case 3 (Obese, Young/Active - MED Expected) Output: 0.5000
Case 4 (High BP, Old, Inactive - HIGH Expected) Output: 0.6705
Case 5 (Triple Threat - VHIGH Expected) Output: 0.8214


In [4]:
# D5 — Add advanced interaction rules to strengthen the system
# These rules capture severe combinations and mitigation patterns
extra_rules = []

# Severe interactions
extra_rules.append(ctrl.Rule(smoker['yes'] & highbp['high'] & physact['inactive'], risk['high']))
extra_rules.append(ctrl.Rule(age['old'] & highbp['high'], risk['vhigh']))
extra_rules.append(ctrl.Rule(bmi['obese'] & smoker['yes'], risk['high']))

# Mitigation patterns
extra_rules.append(ctrl.Rule(bmi['obese'] & age['young'] & physact['active'] & smoker['no'], risk['med']))
extra_rules.append(ctrl.Rule(bmi['healthy'] & age['old'] & physact['active'] & smoker['no'], risk['low']))
extra_rules.append(ctrl.Rule(bmi['over'] & age['old'] & physact['active'] & smoker['no'], risk['med']))

# Emphasis duplicate for critical triple threat
extra_rules.append(ctrl.Rule(bmi['obese'] & highbp['high'] & age['old'], risk['vhigh']))

print(f"Added extra rules: {len(extra_rules)}")
all_rules = all_rules + extra_rules

# Rebuild control system with extended rules
risk_ctrl = ctrl.ControlSystem(all_rules)
risk_sim = ctrl.ControlSystemSimulation(risk_ctrl)
print("Control system rebuilt with extended rule base.")

# Quick sanity sample: severe interaction
print("\n--- Extended Rule Samples ---")
risk_sim.input['BMI'] = 0.7  # obese-ish
risk_sim.input['Age'] = 0.85 # old
risk_sim.input['HighBP'] = 1.0
risk_sim.input['Smoker'] = 1.0
risk_sim.input['PhysActivity'] = 0.0
risk_sim.compute()
print(f"Severe Interaction (Obese+HighBP+Old+Smoker+Inactive) → {risk_sim.output['Risk']:.4f}")

# Mitigation sample
risk_sim.input['BMI'] = 0.8
risk_sim.input['Age'] = 0.2
risk_sim.input['HighBP'] = 0.0
risk_sim.input['Smoker'] = 0.0
risk_sim.input['PhysActivity'] = 1.0
risk_sim.compute()
print(f"Mitigated (Obese+Young+Active+Non-Smoker) → {risk_sim.output['Risk']:.4f}")

Added extra rules: 7
Control system rebuilt with extended rule base.

--- Extended Rule Samples ---
Severe Interaction (Obese+HighBP+Old+Smoker+Inactive) → 0.8119
Mitigated (Obese+Young+Active+Non-Smoker) → 0.5000
